In [15]:
import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2


DATADIR = "D:\projectmachinelearning\Skripsi\images"
CATEGORIES = ["Edible", "Inedible", "Poisonous"]


In [16]:
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                training_data.append([img_array, class_num])
            except Exception as e:
                pass

create_training_data()

In [24]:
print(len(training_data))

661
